## In this time, I applied tf hub for transfer learning

In [1]:
import tensorflow as tf 
from tensorflow import keras

import numpy as np

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(tf.__version__)
print(hub.__version__)
print('GPU', 'On' if tf.config.experimental.list_physical_devices("GPU") else 'Off')

2.3.0
0.12.0
GPU Off


In [2]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

In [3]:
len(train_data), len(validation_data), len(test_data)

(15000, 10000, 25000)

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch # 0: negative review , 1: positive review

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

## Model

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1" # tensorflow hub - bring model that is already made
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:5])

<tf.Tensor: shape=(5, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064],
       [ 1.5165    , -0.71034056,  1.8556767 , -1.2033532 , -1.3105817 ,

In [8]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(64),
                    epochs=20,
                    validation_data=validation_data.batch(64))

Epoch 1/20
235/235 [==============================] - 8s 33ms/step - loss: 0.6696 - accuracy: 0.5857 - val_loss: 0.6442 - val_accuracy: 0.6827
Epoch 2/20
235/235 [==============================] - 7s 31ms/step - loss: 0.6266 - accuracy: 0.7311 - val_loss: 0.6105 - val_accuracy: 0.7821
Epoch 3/20
235/235 [==============================] - 7s 31ms/step - loss: 0.5949 - accuracy: 0.8107 - val_loss: 0.5923 - val_accuracy: 0.8296
Epoch 4/20
235/235 [==============================] - 7s 31ms/step - loss: 0.5742 - accuracy: 0.8557 - val_loss: 0.5789 - val_accuracy: 0.8465
Epoch 5/20
235/235 [==============================] - 7s 31ms/step - loss: 0.5608 - accuracy: 0.8837 - val_loss: 0.5733 - val_accuracy: 0.8597
Epoch 6/20
235/235 [==============================] - 7s 30ms/step - loss: 0.5511 - accuracy: 0.9064 - val_loss: 0.5689 - val_accuracy: 0.8553
Epoch 7/20
235/235 [==============================] - 7s 30ms/step - loss: 0.5437 - accuracy: 0.9213 - val_loss: 0.5676 - val_accuracy: 0.8667

In [11]:
results = model.evaluate(test_data.batch(64))

for metrics, value in zip(model.metrics_names, results):
    print("Metrics: %s Value: %.3f" % (metrics, value))

391/391 [==============================] - 5s 13ms/step - loss: 0.5698 - accuracy: 0.8636
Metrics: loss Value: 0.570
Metrics: accuracy Value: 0.864


## The result is almost same 